#### PASCAL sentence

PASCAL sentenceを http://vision.cs.uiuc.edu/pascal-sentences/ からスクレイピングして保存

画像を224x224にリサイズして保存

In [ ]:
%matplotlib inline
import re
import os
import sys
import urllib
from tqdm import tqdm

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from scipy.misc import imresize

In [ ]:
url = 'http://vision.cs.uiuc.edu/pascal-sentences/'
IMG_DIR = './images'
fp = urllib.request.urlopen(url)
html = str(fp.read())
fp.close()

In [ ]:
def download_img(directory, url):
    img = urllib.request.urlopen(url)
    file_path = os.path.join(directory, os.path.basename(url))
    localfile = open(file_path, 'wb')
    localfile.write(img.read())
    img.close()
    localfile.close()

In [ ]:
img_ptn = r'src=\".*?\.jpg'
img_urls = [url+re.sub(r'src=\"', '', com) for com in re.findall(img_ptn, html)]

for url in tqdm(img_urls):
    download_img(IMG_DIR, url)

In [ ]:
img_paths = [com for com in os.listdir('./images/') if '.jpg' in com]

data = []
for img_path in tqdm(img_paths):
    img = np.array(Image.open(os.path.join(IMG_DIR, img_path)))
    img = imresize(img, (224, 224))
    data.append(img)
data = np.array(data)

np.save('./pascal_images', data)

キャプションを保存

In [ ]:
def tokenize(sentence):
    sentence = re.sub(r'\.', ' .', sentence)
    sentence = re.sub(r',', ' ,', sentence)
    sentence = re.sub(r'\'s', ' \'s', sentence)
    return '<s> ' + sentence + ' </s>'

In [ ]:
cap_ptn = re.compile(r'\<td\>\<table\>\n\<tr\>.*?\<\/td\>', re.MULTILINE|re.DOTALL)
captions = [re.sub(r'(<.*?>|\n)', '', com)[1:] for com in re.findall(cap_ptn, html)]
captions = '\n'.join([tokenize(com) for com in captions])

with open('./pascal_captions.txt', 'w') as f:
    f.write(captions)

#### MS COCO

In [1]:
import json
from pycocotools.coco import COCO

import re
import os
import sys
import urllib
from tqdm import tqdm

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from scipy.misc import imresize
from nltk.tokenize import word_tokenize

IMG_DIR = '/home/fujino/ail/train2014/'

In [2]:
data = json.load(open('/home/fujino/ail/annotations/captions_train2014.json'))

In [3]:
# initialize COCO api for caption annotations
annFile = '/home/fujino/ail/annotations/captions_train2014.json'
coco_caps=COCO(annFile)

loading annotations into memory...
Done (t=1.16s)
creating index...
index created!


In [4]:
def get_resized_image(file_name):
    image = np.array(Image.open(os.path.join(IMG_DIR, file_name)))
    return imresize(image, (224, 224))

In [5]:
captions = ''
images = []

for datum in tqdm(data['images']):
    # Image
    image = get_resized_image(datum['file_name'])
    if image.shape != (224, 224, 3):
        continue
    images.append(image)

    # Caption
    annIds = coco_caps.getAnnIds(imgIds=datum['id'])
    anns = coco_caps.loadAnns(annIds)
    caption = anns[0]['caption']
    captions += '<s> ' + ' '.join(word_tokenize(caption.lower())) + ' </s>\n'

100%|██████████| 82783/82783 [23:55<00:00, 57.68it/s]  


In [ ]:
# mscoco_images = np.array(images)

In [ ]:
# np.save('mscoco_images', mscoco_images)

In [6]:
with open('mscoco_captions.txt', 'w') as f:
    f.write(captions)